<a href="https://colab.research.google.com/github/NapoliD/telecom/blob/master/Clientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import os
# Ejemplo si el archivo esta cargado en el entorno actual
os.chdir('/content/')

In [ ]:
import numpy as np
import pandas as pd


<h1>1-Crear una marca en caso de que haya cambiado de plan en alguno de los tres meses</h1>

In [104]:
datos = pd.read_csv('cliente.csv', sep= ',')
datos

,Plan_mes,mes,nro_cliente
0,95729,1,1
1,96410,1,3
2,96400,1,2
3,95729,1,5
4,95420,1,4
...,...,...,...
32613,96806,3,10897
32614,96890,3,10895
32615,96806,3,10898
32616,96806,3,10900


In [105]:
clientes=datos.groupby('nro_cliente')['Plan_mes'].aggregate(lambda x: x.tolist()).reset_index()
clientes

,nro_cliente,Plan_mes
0,1,"[95729, 95729, 96810]"
1,2,"[96400, 96400, 96400]"
2,3,"[96410, 96410, 96410]"
3,4,"[95420, 96413, 96413]"
4,5,"[95729, 96810, 96810]"
...,...,...
10895,10896,"[96806, 96806]"
10896,10897,"[96806, 96806]"
10897,10898,"[96806, 96806]"
10898,10899,"[96766, 96770]"


In [108]:
planes=clientes['Plan_mes']
planes=pd.DataFrame(planes).astype(str)
planes=pd.DataFrame(planes['Plan_mes'].apply(lambda x: pd.Series(x.split(','))))
planes

,0,1,2
0,[95729,95729,96810]
1,[96400,96400,96400]
2,[96410,96410,96410]
3,[95420,96413,96413]
4,[95729,96810,96810]
...,...,...,...
10895,[96806,96806],NaN
10896,[96806,96806],NaN
10897,[96806,96806],NaN
10898,[96766,96770],NaN


In [109]:
mes1=pd.DataFrame(planes[0].map(lambda x: x.lstrip('[')).astype(int))
mes1.columns=['mes1']
mes1

,mes1
0,95729
1,96400
2,96410
3,95420
4,95729
...,...
10895,96806
10896,96806
10897,96806
10898,96766


In [110]:
mes2=planes[1].astype(str).apply(lambda x: pd.Series(x.split(']')))
mes2=pd.DataFrame(mes2[0].astype(int))
mes2.columns=['mes2']
mes2

,mes2
0,95729
1,96400
2,96410
3,96413
4,96810
...,...
10895,96806
10896,96806
10897,96806
10898,96770


In [111]:
mes3=planes[2].astype(str).apply(lambda x: pd.Series(x.split(']')))
mes3=pd.DataFrame(mes3[0].replace('nan',0).astype(int))
mes3.columns=['mes3']
mes3

,mes3
0,96810
1,96400
2,96410
3,96413
4,96810
...,...
10895,0
10896,0
10897,0
10898,0


In [112]:
base_final=pd.concat([clientes.reset_index(drop=True),
                      mes1.reset_index(drop=True),mes2.reset_index(drop=True),mes3.reset_index(drop=True)], axis=1)
base_final

In [ ]:
base_final.drop_duplicates(subset='nro_cliente',keep='first')

In [114]:
base_final['cambio_plan']=np.where( (base_final['mes1']!=base_final['mes2']),1,0)
base_final['cambio_plan1']=np.where((base_final['mes1']!=base_final['mes3']) & (base_final['mes3']!=0),1,0)
base_final['cambio_plan2']=np.where( (base_final['mes2']!=base_final['mes3']) &  (base_final['mes3']!=0),1,0)
base_final['cambio_plan_1_si_0_no']=np.where( (base_final['cambio_plan']==1) | (base_final['cambio_plan1']==1) | (base_final['cambio_plan2']==1),1,0)
base_final

,nro_cliente,Plan_mes,mes1,mes2,mes3,cambio_plan,cambio_plan1,cambio_plan2,cambio_plan_1_si_0_no
0,1,"[95729, 95729, 96810]",95729,95729,96810,0,1,1,1
1,2,"[96400, 96400, 96400]",96400,96400,96400,0,0,0,0
2,3,"[96410, 96410, 96410]",96410,96410,96410,0,0,0,0
3,4,"[95420, 96413, 96413]",95420,96413,96413,1,1,0,1
4,5,"[95729, 96810, 96810]",95729,96810,96810,1,1,0,1
...,...,...,...,...,...,...,...,...,...
10895,10896,"[96806, 96806]",96806,96806,0,0,0,0,0
10896,10897,"[96806, 96806]",96806,96806,0,0,0,0,0
10897,10898,"[96806, 96806]",96806,96806,0,0,0,0,0
10898,10899,"[96766, 96770]",96766,96770,0,1,0,0,1


In [133]:
base_final.loc[base_final['cambio_plan_1_si_0_no']==1].count()

nro_cliente              5847
Plan_mes                 5847
mes1                     5847
mes2                     5847
mes3                     5847
cambio_plan              5847
cambio_plan1             5847
cambio_plan2             5847
cambio_plan_1_si_0_no    5847
dtype: int64

In [115]:
base_final2=base_final.loc[:,['nro_cliente','cambio_plan_1_si_0_no']]
base_final_clientes=datos.merge(base_final2, on='nro_cliente', how='left')
base_final_clientes


,Plan_mes,mes,nro_cliente,cambio_plan_1_si_0_no
0,95729,1,1,1
1,96410,1,3,0
2,96400,1,2,0
3,95729,1,5,1
4,95420,1,4,1
...,...,...,...,...
32613,96806,3,10897,0
32614,96890,3,10895,1
32615,96806,3,10898,0
32616,96806,3,10900,0


In [135]:
base_final_clientes.to_csv("Clientes_cambio_plan.csv",index=False)

<h1>2- Cuantos clientes que se encuentran en CABA cambiaron de plan.</h1>

In [119]:
provincia = pd.read_csv('provincia.csv',sep=';',encoding='latin-1',decimal=',',thousands='.')
provincia

,provincia,nro_cliente
0,Chaco,1
1,Santa Fe,2
2,Córdoba,3
3,Salta,4
4,Santa Fe,5
...,...,...
10895,Catamarca,10896
10896,Buenos Aires,10897
10897,San Juan,10898
10898,Chaco,10899


In [120]:
provincia2=base_final2.merge(provincia, on='nro_cliente', how='left')
provincia2

,nro_cliente,cambio_plan_1_si_0_no,provincia
0,1,1,Chaco
1,2,0,Santa Fe
2,3,0,Córdoba
3,4,1,Salta
4,5,1,Santa Fe
...,...,...,...
10895,10896,0,Catamarca
10896,10897,0,Buenos Aires
10897,10898,0,San Juan
10898,10899,1,Chaco


In [121]:
cambiaron_caba=provincia2[(provincia2['cambio_plan_1_si_0_no']==1) & (provincia2['provincia'].str.contains("Ciudad Aut"))]
cant_camb=cambiaron_caba['nro_cliente'].count()
print(f'La cantidad de clientes que cambiaron de plan y encuentran en CABA son: {cant_camb} clientes')

La cantidad de clientes que cambiaron de plan y encuentran en CABA son: 706 clientes


In [134]:
cambiaron_caba.to_csv("Cambiaron_de_plan_caba.csv",index=False)

<h1>3- Que provincia tiene el promedio mensual de facturación más alta</h1>


In [123]:
facturacion = pd.read_csv('facturacion.csv',sep=';',encoding='latin-1',decimal=',',thousands='.')
facturacion=facturacion.merge(provincia, on='nro_cliente', how='left')
facturacion

,facturacion,nro_cliente,mes,provincia
0,9358301,1,1,Chaco
1,13528632,2,1,Santa Fe
2,6020372,3,1,Córdoba
3,8475758,4,1,Salta
4,423793,5,1,Santa Fe
...,...,...,...,...
32695,5662178,10896,3,Catamarca
32696,799,10897,3,Buenos Aires
32697,0,10898,3,San Juan
32698,0,10899,3,Chaco


In [124]:
facturacion2=pd.DataFrame(facturacion.groupby(['provincia'])['facturacion'].sum().astype(int)).sort_values(by=['facturacion'],ascending=False)
provincua_con_mas=pd.DataFrame((facturacion2['facturacion']/3).astype(int))
provincua_con_mas.head(1)

,facturacion
provincia,
Buenos Aires,12638233493


In [136]:
provincua_con_mas.to_csv("provincia_con_mas_factu.csv",index=False)